# Data Preparation
- Data set Los Angeles 2019

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import data set
df_la = pd.read_csv("la_2019.csv", parse_dates=["start_time","end_time"], low_memory=False)

In [3]:
# data overview
df_la.head(3)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th


In [4]:
print(df_la.count())
print(df_la.info())

start_time            290342
end_time              290342
start_station_id      290342
end_station_id        290342
bike_id               290342
user_type             290342
start_station_name    290342
end_station_name      290342
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290342 entries, 0 to 290341
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   start_time          290342 non-null  datetime64[ns]
 1   end_time            290342 non-null  datetime64[ns]
 2   start_station_id    290342 non-null  int64         
 3   end_station_id      290342 non-null  int64         
 4   bike_id             290342 non-null  object        
 5   user_type           290342 non-null  object        
 6   start_station_name  290342 non-null  object        
 7   end_station_name    290342 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 17.7+ MB
None


The Los Angeles data set includes bike rental data for the period 01/01/2019 to 12/31/2019. The data set has 290.342 entries.

### Clean the data set

In [5]:
# check if there are null values
df_la.isnull().any()

start_time            False
end_time              False
start_station_id      False
end_station_id        False
bike_id               False
user_type             False
start_station_name    False
end_station_name      False
dtype: bool

In [6]:
# drop rows which contain missing values
df_la.dropna(axis=0, inplace=True)

In [7]:
df_la["bike_id"].size

290342

In [8]:
# add the duration of the trip
df_la["duration"] = (df_la["end_time"] - df_la["start_time"]).astype("timedelta64[m]")

In [9]:
# remove data with same start and end station, that are shorter or equal to one minute
df_la = df_la[~((df_la["start_station_name"] == df_la["end_station_name"]) & (df_la["duration"] <= 1.0))]

In [10]:
df_la["bike_id"].size

284417

In [11]:
# remove trips that are longer or equal to 24h
df_la = df_la[ ~ (df_la["duration"] >= 1440.0) ]

In [12]:
df_la["bike_id"].size

283135

In [13]:
df_la.drop_duplicates()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway,7.0
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,6.0
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,05860,Walk-up,Main & 1st,Broadway & 9th,30.0
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,06006,Walk-up,Main & 1st,Broadway & 9th,28.0
...,...,...,...,...,...,...,...,...,...
290336,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,3069,12019,Walk-up,Pershing Square,Broadway & 3rd,7.0
290338,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe,9.0
290339,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th,4.0
290340,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic,5.0


In [14]:
df_la["bike_id"].size

283135

In [15]:
# unnecessary as we use start/end_station_name
df_la = df_la.drop(["start_station_id"], axis=1)
df_la = df_la.drop(["end_station_id"], axis=1)

### Adding useful columns

In [16]:
# new columns 
df_la["date"] = df_la["start_time"].apply(lambda x: x.date())
df_la['month']= df_la["start_time"].apply(lambda x: x.month)
df_la["weekday"]= df_la["start_time"].apply(lambda x: x.weekday())
df_la["day"] = df_la["start_time"].apply(lambda x: x.day)
df_la["hour"]= df_la["start_time"].apply(lambda x: x.hour)
df_la.head(3)

,start_time,end_time,bike_id,user_type,start_station_name,end_station_name,duration,date,month,weekday,day,hour
0,2019-01-01 00:07:00,2019-01-01 00:14:00,06468,Walk-up,2nd & Hill,7th & Broadway,7.0,2019-01-01,1,1,1,0
1,2019-01-01 00:08:00,2019-01-01 00:14:00,12311,Walk-up,2nd & Hill,7th & Broadway,6.0,2019-01-01,1,1,1,0
2,2019-01-01 00:18:00,2019-01-01 00:50:00,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,2019-01-01,1,1,1,0


In [17]:
# saving data set
df_la.to_csv("prepared_data.csv")

## Weather Data 

In [18]:
df_weather = pd.read_csv("weather_hourly_la.csv" ,parse_dates=["date_time"])
df_weather.head(3)

,date_time,max_temp,min_temp,precip
0,2015-01-02 01:00:00,11.7,11.7,0.0
1,2015-01-02 02:00:00,11.1,11.1,0.0
2,2015-01-02 03:00:00,11.1,11.1,0.0


In [19]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  43756 non-null  datetime64[ns]
 1   max_temp   43756 non-null  float64       
 2   min_temp   43756 non-null  float64       
 3   precip     43758 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 1.3 MB


In [20]:
# keep only data from 2019
df_weather = df_weather[df_weather["date_time"].apply(lambda x: x.year == 2019)]
df_weather = df_weather.dropna()
df_weather.head(3)

,date_time,max_temp,min_temp,precip
35040,2019-01-01 01:00:00,15.6,15.6,0.0
35041,2019-01-01 02:00:00,15.0,15.0,0.0
35042,2019-01-01 03:00:00,15.0,15.0,0.0


In [21]:
df_weather["date_time"].size

8732

In [22]:
df_weather.isnull().any()
df_weather.dropna(axis=0, inplace=True)

In [23]:
df_weather["date_time"].size

8732

In [24]:
df_weather.drop_duplicates()

,date_time,max_temp,min_temp,precip
35040,2019-01-01 01:00:00,15.6,15.6,0.0
35041,2019-01-01 02:00:00,15.0,15.0,0.0
35042,2019-01-01 03:00:00,15.0,15.0,0.0
35043,2019-01-01 04:00:00,13.9,13.9,0.0
35044,2019-01-01 05:00:00,12.2,12.2,0.0
...,...,...,...,...
43818,2019-12-31 19:00:00,20.0,20.0,0.0
43819,2019-12-31 20:00:00,20.6,20.6,0.0
43820,2019-12-31 21:00:00,21.1,21.1,0.0
43821,2019-12-31 22:00:00,21.7,21.7,0.0


In [25]:
df_weather["date_time"].size

8732

In [26]:
df_weather = df_weather.drop(["min_temp"], axis=1)

In [27]:
df_weather["precip"].value_counts()

0.0    8476
1.0     256
Name: precip, dtype: int64

In [28]:
# add date, month, day, hour
df_weather["date"] = df_weather["date_time"].apply(lambda x:x.date())
df_weather["month"] = df_weather["date_time"].apply(lambda x:x.month)
df_weather["day"] = df_weather["date_time"].apply(lambda x:x.day)
df_weather["hour"] = df_weather["date_time"].apply(lambda x:x.hour)
df_weather.sort_values(by="date_time")
df_weather.head(1)

,date_time,max_temp,precip,date,month,day,hour
35040,2019-01-01 01:00:00,15.6,0.0,2019-01-01,1,1,1


In [29]:
# saving data set
df_weather.to_csv("prepared_data_weather.csv")

In [30]:
# merge weather and bikesharing data
merged_data = df_la.merge(df_weather[["max_temp","precip","date","month","day","hour"]],left_on=["date","month","day","hour"],right_on=["date","month","day","hour"])
merged_data.head(1)

,start_time,end_time,bike_id,user_type,start_station_name,end_station_name,duration,date,month,weekday,day,hour,max_temp,precip
0,2019-01-01 00:07:00,2019-01-01 00:14:00,06468,Walk-up,2nd & Hill,7th & Broadway,7.0,2019-01-01,1,1,1,0,15.6,0.0


In [31]:
# saving data set
merged_data.to_csv("merged_data.csv")

## Dataset for modelling

Possible features:
1. Month/Season
2. Day
3. Hour
4. Duration
5. Temperature
6. Precipitation
7. Weekday/Weekend

In [32]:
# pick feature relevant columns
feature_data = merged_data[["start_time","month","day","hour","duration","max_temp","precip","bike_id"]]
feature_data.sort_values(by="start_time").head(1)

,start_time,month,day,hour,duration,max_temp,precip,bike_id
0,2019-01-01 00:07:00,1,1,0,7.0,15.6,0.0,06468


In [33]:
# aggregate the samples into hourly chunks
feature_data = feature_data.groupby(pd.Grouper(key='start_time',freq='1H')).agg({"month":"mean","day":"mean","hour":"mean","duration":"mean","max_temp":"mean","precip":"mean","bike_id":"count"})
feature_data.rename(columns={"duration":"mean_duration","bike_id":"trips"}, inplace=True)
# deal with null values
feature_data.loc[feature_data["month"].isna(),"month"] = feature_data.index.to_series().apply(lambda x: x.month)
feature_data.loc[feature_data["day"].isna(),"day"] = feature_data.index.to_series().apply(lambda x: x.day)
feature_data.loc[feature_data["hour"].isna(),"hour"] = feature_data.index.to_series().apply(lambda x: x.hour)
feature_data = feature_data.fillna(method="ffill")
feature_data.head()

,month,day,hour,mean_duration,max_temp,precip,trips
start_time,,,,,,,
2019-01-01 00:00:00,1.0,1.0,0.0,15.736842,15.6,0.0,19
2019-01-01 01:00:00,1.0,1.0,1.0,34.625000,15.6,0.0,8
2019-01-01 02:00:00,1.0,1.0,2.0,45.812500,15.0,0.0,16
2019-01-01 03:00:00,1.0,1.0,3.0,30.000000,15.0,0.0,2
2019-01-01 04:00:00,1.0,1.0,4.0,30.000000,15.0,0.0,0


In [34]:
# add weekday feature
feature_data["isWeekday"] = feature_data.index.to_series().apply(lambda x: 1 if x.dayofweek in [5,6] else 0)
feature_data.head()

,month,day,hour,mean_duration,max_temp,precip,trips,isWeekday
start_time,,,,,,,,
2019-01-01 00:00:00,1.0,1.0,0.0,15.736842,15.6,0.0,19,0
2019-01-01 01:00:00,1.0,1.0,1.0,34.625000,15.6,0.0,8,0
2019-01-01 02:00:00,1.0,1.0,2.0,45.812500,15.0,0.0,16,0
2019-01-01 03:00:00,1.0,1.0,3.0,30.000000,15.0,0.0,2,0
2019-01-01 04:00:00,1.0,1.0,4.0,30.000000,15.0,0.0,0,0


In [35]:
#convert to csv
feature_data.to_csv('prediction_data.csv')